In [255]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score

import xgboost as xgb

import optuna

pd.set_option('display.max_columns', 200)

In [155]:
data = pd.read_csv('intern_task.csv')

In [156]:
data = data.rename(columns={'rank': 'score'})

# Задача ранжирования

Что подразумевается под задачей ранжирования? Представим, что в обучающей выборке есть какое-то множество запросов, для которых известен порядок документов по релевантности. Например, вы знаете, какой документ самый релевантный, какой второй по релевантности и т.д. И вам нужно восстановить такой порядок для всей генеральной совокупности. То есть для всех запросов из генеральной совокупности на первое место поставить самый релевантный документ, а на последнее — самый нерелевантный.

In [157]:
data['score'].unique()

array([0, 1, 2, 4, 3], dtype=int64)

Каждая строка датасета представляет собой пару "запрос-документ", и ранг документа (оценка релевантности) принимает значение от 0 до 4, где 0 означает "документ полностью нерелевантен запроса", а 4 - "документ полностью релевантен запросу".

# Подготовка данных
Поскольку имеем задачу ранжирования, вместо привычного train_test_split будем использовать GroupShuffleSplit, чтобы при делении датасета на train и test выборки не смешивать документы из разных запросов

In [162]:
group_ss = GroupShuffleSplit(test_size=0.20, n_splits=1, random_state = 42).split(data, groups=data['query_id'])

In [163]:
train_idx, test_idx = next(group_ss)

In [164]:
train_data = data.iloc[train_idx]
test_data = data.iloc[test_idx]

In [223]:
X_train = train_data.drop(columns=['score'])
y_train = train_data['score']

In [224]:
groups_train = train_data.groupby('query_id').size().values # для последующего обучения XGBRanker нам нужны размеры выборок, сгруппированных по query_id

In [225]:
X_test = test_data.drop(columns=['score'])
y_test = test_data['score']

groups_test = test_data.groupby('query_id').size().values

Как оценивать модель? (т.е. какой loss выбрать?). Статья от VK - https://habr.com/ru/companies/vk/articles/461927/ предлагает 3 подхода:  
**Pointwise**, он же поточечный. Мы будем рассматривать релевантность как абсолютное мерило и будем штрафовать модель за абсолютную разность между предсказанной релевантностью и той, которую мы знаем по обучающей выборке. Например, асессор поставил документу оценку 3, а мы бы сказали 2, поэтому штрафуем модель на 1.  
**Pairwise**, попарный. Мы будем сравнивать документы друг с другом. Например, в обучающей выборке есть два документа, и нам известно, какой из них более релевантный по данному запроса. Тогда мы будем штрафовать модель, если она более релевантному поставила прогноз ниже, чем менее релевантному, то есть неправильно сранжировала пару.  
**Listwise**. Он тоже основан на относительных релевантностях, но уже не внутри пар: мы ранжируем моделью всю выдачу и оцениваем результат — если на первом месте оказался не самый релевантный документ, то получаем большой штраф.  
Согласно статье, использовать лучше всего pairwise, либо listwise. Придержимся первого подхода, и отразится он в параметре **objective** модели **XGBRanker**. XGBoost предоставляет несколько loss-функций для этого подхода, одной из которых является rank:ndcg. Очень удобно, учитывая метрику, по которой оценивается модель

In [226]:
ranker = xgb.XGBRanker(
    objective='rank:ndcg',
    random_state=42,
    tree_method='hist',
    booster='gbtree',
    learning_rate=0.1,
    colsample_bytree=0.9,
    max_depth=6,
    n_estimators=1000,
    subsample=0.75,
    reg_lambda=0.05,
    early_stopping_rounds=100
)

In [227]:
ranker.fit(
    X_train,
    y_train,
    group=groups_train,
    eval_group=[groups_test],
    eval_set=[(X_test, y_test)],
    verbose = 100
)

[0]	validation_0-ndcg@32:0.47620


C:\Users\user\anaconda3\Lib\site-packages\xgboost\sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-ndcg@32:0.55632
[200]	validation_0-ndcg@32:0.56164
[281]	validation_0-ndcg@32:0.56341


XGBRanker(base_score=None, booster='gbtree', callbacks=None,
          colsample_bylevel=None, colsample_bynode=None, colsample_bytree=0.9,
          device=None, early_stopping_rounds=None, enable_categorical=False,
          eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
          importance_type=None, interaction_constraints=None, learning_rate=0.1,
          max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
          max_delta_step=None, max_depth=6, max_leaves=None,
          min_child_weight=None, missing=nan, monotone_constraints=None,
          multi_strategy=None, n_estimators=1000, n_jobs=None,
          num_parallel_tree=None, random_state=42, ...)

После обучения модели, хотелось бы как-то интерпретировать результаты ее работы. В задаче ранжирования нам было бы интересно посмотреть на релевантность документов в рамках отдельного запроса (query_id)

In [238]:
test_data[test_data.query_id == 29860]

,score,query_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143
234345,1,29860,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,996.123388,0.2,11.0,0.0,3.0,4.0,18.0,16.813771,39.355411,35.8821,38.815724,16.808963,0.996123,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.992247e+06,0.0,0.0,0.2,0.2,0.00,0.00,0.00,0.16,0.16,0.000000,0.000000,0.000000,2.054943,0.055556,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.055556,0.000000,0.000000,0.000000,0.050000,0,0,0.000000,0.000000,0.010000,0.000494,0.000000,0.000000,1,8.246179,3.695372,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,8.246179,3.695372,0.000000,0.000000,0.000000,1.649236,0.739074,0.000000,0.000000,0.000000,10.879915,2.184924,0.0,0.0,0.0,0.0,0.0,0,0.000000,0.000000,0.468599,0.451533,0.000000,0.000000,0.000000,11.867319,8.386458,-37.967632,-41.764541,-43.493183,-43.056846,-43.773123,-38.655990,-43.670926,-45.081440,-46.591206,-44.789194,-38.575818,-48.464156,-49.570364,-49.742149,-49.454592,1.0,22.0,0.0,0.0,120.0,416.0,246.0,78.0,0.0,0.0,0.000000,0.250000,0.000000,0.000000,0.2,0.011111,0.000000,0.0,0.0
234346,2,29860,5.0,0.0,4.0,0.0,5.0,1.0,0.0,0.8,996.127639,1.0,38.0,0.0,6.0,7.0,51.0,16.813771,39.355411,35.8821,38.815724,16.808963,0.996128,0.0,4.0,0.0,14.0,1.0,0.0,0.0,0.0,1.0,3.0,0.0,1.0,0.0,4.0,1.992255e+06,0.0,0.8,0.0,2.8,0.40,0.00,0.16,0.00,0.96,0.263158,0.000000,0.666667,4.873028,0.274510,0.026316,0.0,0.0,0.000000,0.019608,0.078947,0.000000,0.166667,0.000000,0.078431,0.052632,0.000000,0.133333,0.000000,0,0,0.000000,0.004444,0.000000,0.000369,33.734551,0.000000,1,0.000000,48.860706,1.674283,0.0,0.0,0.000000,1.673131,10.021419,0.000000,8.699208,0.000000,15.029748,6.746910,0.000000,6.066891,0.000000,9.772141,9.074186,0.000000,10.601949,0.000000,23.892247,1.0,0.0,0.0,0.0,1.0,0,0.000000,0.940728,0.000000,0.985774,44.976771,0.000000,34.141937,0.000000,47.753160,-17.737336,-41.764541,-19.806666,-46.473643,-17.134180,-33.917040,-43.670926,-36.831791,-48.415677,-32.533827,-15.533922,-48.464156,-17.368478,-53.167057,-15.500400,3.0,37.0,1.0,0.0,124.0,3407.0,19.0,9.0,0.0,0.0,0.000000,0.000000,0.987141,30.334453,0.0,0.054902,0.000277,10.0,2.0
234347,1,29860,5.0,0.0,1.0,1.0,5.0,1.0,0.0,0.2,996.131890,1.0,871.0,0.0,6.0,7.0,884.0,16.813771,39.355411,35.8821,38.815724,16.808963,0.996132,0.0,1.0,1.0,85.0,3.0,0.0,0.0,0.0,3.0,53.0,0.0,1.0,1.0,53.0,1.992264e+06,0.0,0.2,0.2,17.0,345.04,0.00,0.16,0.16,343.60,0.095293

In [233]:
test_query = test_data[test_data.query_id == 29860].drop(columns=['score'])

Допустим, мы хотим получить предсказание для последнего запроса в test-выборке

In [235]:
y_pred = ranker.predict(test_query)

Получили numpy-массив, со скорами документов: чем выше скор, тем выше релевантность документа запросу.  
Теперь нам бы хотелось оценить качество модели ранжирования метрикой ndcg. Следующий код был честно позаимствован из: https://github.com/catboost/benchmarks/tree/master/ranking#metric

In [266]:
def dcg(y_score, y_true, k):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)

Теперь надо получить список уникальных query_id из test-выборки. Мы будем циклом проходиться по ним, для каждого query_id вычислять dcg-скор и добавлять этот скор в list, а в конце возьмем среднее по получившемуся списку - и по итогу получим **N**(normalized)**DCG**

In [261]:
query_ids = test_data.query_id.unique()

In [270]:
def nDCG(data, ids, model):
    dcg_list = list()

    for i, q_id in enumerate(ids):
        X = data[data.query_id == q_id].drop(columns=['score'])
        y = data[data.query_id == q_id]['score']

        if np.sum(y) == 0:  # для случая нерелевантности всех документов из запроса
            continue

        preds = model.predict(X)

        idcg = dcg(y, y, k=5) # идеальный вариант DCG

        dcg_ = dcg(preds, y, k=5) / idcg # k=5 - выполняем требование по тз :)

        dcg_list.append(dcg_)
        
    return np.mean(dcg_list)

In [271]:
nDCG(test_data, query_ids, ranker)

0.471454906761453

После построения "бейзлайна" можно попробовать потюнить модель через optuna

In [282]:
def objective_XGBRanker(trial):
    
    params = {
         'n_estimators': trial.suggest_int('iterations', 1000, 2000, step=200),
         'max_depth': trial.suggest_int('depth', 3, 9),
         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1 , step=0.01),
         'objective': 'rank:ndcg',
         'tree_method': "hist",
         'booster': 'gbtree',
         'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0, step=0.1),
         'subsample': trial.suggest_float('subsample', 0.1, 1.0, step=0.05),
         'reg_lambda': trial.suggest_float('reg_labmda', 0.01, 0.2, step=0.02)
    }
    
    ranker = xgb.XGBRanker(**params,
                           early_stopping_rounds=100,
                           random_seed=42)
    
    ranker.fit(
    X_train,
    y_train,
    group=groups_train,
    eval_group=[groups_test],
    eval_set=[(X_test, y_test)],
    verbose = False
    )
    
    query_ids = test_data.query_id.unique()
    
    return nDCG(test_data, query_ids, ranker)

In [283]:
study_ranker = optuna.create_study(study_name='xgbranker-seed42',
                                direction='maximize')

In [284]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_ranker.optimize(objective_XGBRanker, n_trials=10,show_progress_bar=True)

  0%|          | 0/10 [00:00<?, ?it/s]

C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:13:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and step=0.02, but the range is not divisible by `step`. It will be replaced by [0.01, 0.19].
  warnings.warn(
C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:14:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07f6e447eee219473-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\user\anaconda3\Lib\site-packages\optuna\distributions.py:685: UserWarning: The distribution is specified by [0.01, 0.2] and ste

Лучший скор **NDCG_5** - **0.482854** (k=5)